In [1]:
!pip install nltk meteor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install git+https://github.com/salaniz/pycocoevalcap.git

  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-wwzzqt8n
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-wwzzqt8n
  Resolved https://github.com/salaniz/pycocoevalcap.git to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=8377ebcde25b07cb6714c2aec8519624c227774485edc3f0aa1c41d9d7aee570
  Stored in directory: /tmp/pip-ephem-wheel-cache-snez6tb1/wheels/b7/7d/bd/ee763c30e9e11deccc65814eb10c3f58b155fee1d68c0a392f
Successfully built pycocoevalcap


In [2]:
!pip install -q nltk evaluate bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.9 MB/s eta 0:00:00


In [3]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from evaluate import load
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [4]:
import json

In [5]:
def calculate_bleu(reference, hypothesis):
    """Calcula BLEU score"""
    ref_tokens = [reference.lower().split()]
    hyp_tokens = hypothesis.lower().split()
    smooth = SmoothingFunction().method1
    return sentence_bleu(ref_tokens, hyp_tokens, smoothing_function=smooth)

def calculate_meteor(reference, hypothesis):
    """Calcula METEOR score"""
    meteor = load('meteor')
    score = meteor.compute(predictions=[hypothesis], references=[[reference]])
    return score['meteor']

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Cargar las anotaciones y los resultados generados
annotations_path = '/content/drive/MyDrive/Imagenes Proyecto/annotations.json'
results_IC_path = '/content/drive/MyDrive/Imagenes Proyecto/results_florence2_IC_only.json'

with open(annotations_path, 'r') as f:
    annotations = json.load(f)

with open(results_IC_path, 'r') as f:
    results = json.load(f)

In [8]:
print("="*70)
print("CALCULANDO MÉTRICAS")
print("="*70 + "\n")

# Mapeo de prompts a niveles
prompt_to_level = {
    "<CAPTION>": "basic",
    "<DETAILED_CAPTION>": "intermediate",
    "<MORE_DETAILED_CAPTION>": "advanced"
}

# Almacenar todas las métricas
all_metrics = []
metrics_by_level = {
    "basic": {"bleu": [], "meteor": []},
    "intermediate": {"bleu": [], "meteor": []},
    "advanced": {"bleu": [], "meteor": []}
}

# Procesar cada imagen
for image_data in results:
    image_name = image_data["image_path"]
    annotations = image_data["annotations"]  # [basic, intermediate, advanced]
    prompt_outputs = image_data["prompt_outputs"]

    print(f"{image_name}")

    image_metrics = {
        "image": image_name,
        "levels": {}
    }

    # Procesar cada nivel
    for i, output_data in enumerate(prompt_outputs):
        prompt = output_data["prompt"]
        generated = output_data["output"]
        level = prompt_to_level[prompt]
        reference = annotations[i]  # Mismo índice que el prompt

        # Calcular métricas
        try:
            bleu = calculate_bleu(reference, generated)
            meteor = calculate_meteor(reference, generated)

            # Guardar
            image_metrics["levels"][level] = {
                "reference": reference,
                "generated": generated,
                "bleu": round(bleu, 4),
                "meteor": round(meteor, 4)
            }

            # Agregar a estadísticas
            metrics_by_level[level]["bleu"].append(bleu)
            metrics_by_level[level]["meteor"].append(meteor)

            print(f"  {level:12} | BLEU: {bleu:.4f} | METEOR: {meteor:.4f}")

        except Exception as e:
            print(f" Error en {level}: {e}")

    all_metrics.append(image_metrics)
    print()

CALCULANDO MÉTRICAS

01.jpg


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


  basic        | BLEU: 0.0561 | METEOR: 0.6899


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


...

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


  basic        | BLEU: 0.0204 | METEOR: 0.2655


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


  intermediate | BLEU: 0.0046 | METEOR: 0.1166
  advanced     | BLEU: 0.0304 | METEOR: 0.1549



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [9]:
import numpy as np

In [10]:
print("="*70)
print("PROMEDIOS POR NIVEL")
print("="*70 + "\n")

summary = {
    "model": "Florence-2-base-ft",
    "total_images": len(results),
    "metrics_by_level": {},
    "overall": {}
}

for level in ["basic", "intermediate", "advanced"]:
    bleu_scores = metrics_by_level[level]["bleu"]
    meteor_scores = metrics_by_level[level]["meteor"]

    if bleu_scores:
        avg_bleu = np.mean(bleu_scores)
        avg_meteor = np.mean(meteor_scores)
        std_bleu = np.std(bleu_scores)
        std_meteor = np.std(meteor_scores)

        summary["metrics_by_level"][level] = {
            "bleu": {
                "mean": round(avg_bleu, 4),
                "std": round(std_bleu, 4),
                "min": round(min(bleu_scores), 4),
                "max": round(max(bleu_scores), 4)
            },
            "meteor": {
                "mean": round(avg_meteor, 4),
                "std": round(std_meteor, 4),
                "min": round(min(meteor_scores), 4),
                "max": round(max(meteor_scores), 4)
            },
            "samples": len(bleu_scores)
        }

        print(f"{level.upper()}")
        print(f"   BLEU:   {avg_bleu:.4f} ± {std_bleu:.4f}")
        print(f"   METEOR: {avg_meteor:.4f} ± {std_meteor:.4f}")
        print(f"   Rango BLEU:   [{min(bleu_scores):.4f}, {max(bleu_scores):.4f}]")
        print(f"   Rango METEOR: [{min(meteor_scores):.4f}, {max(meteor_scores):.4f}]")
        print(f"   Muestras: {len(bleu_scores)}\n")

# Promedio general
all_bleu = []
all_meteor = []
for level_data in metrics_by_level.values():
    all_bleu.extend(level_data["bleu"])
    all_meteor.extend(level_data["meteor"])

summary["overall"] = {
    "bleu": {
        "mean": round(np.mean(all_bleu), 4),
        "std": round(np.std(all_bleu), 4)
    },
    "meteor": {
        "mean": round(np.mean(all_meteor), 4),
        "std": round(np.std(all_meteor), 4)
    },
    "total_samples": len(all_bleu)
}

print("="*70)
print("PROMEDIO GENERAL (TODOS LOS NIVELES)")
print("="*70)
print(f"BLEU:   {np.mean(all_bleu):.4f} ± {np.std(all_bleu):.4f}")
print(f"METEOR: {np.mean(all_meteor):.4f} ± {np.std(all_meteor):.4f}")
print(f"Total: {len(all_bleu)} muestras")
print("="*70 + "\n")

PROMEDIOS POR NIVEL

BASIC
   BLEU:   0.0638 ± 0.0929
   METEOR: 0.3775 ± 0.2034
   Rango BLEU:   [0.0204, 0.4029]
   Rango METEOR: [0.1271, 0.7634]
   Muestras: 15

INTERMEDIATE
   BLEU:   0.0122 ± 0.0051
   METEOR: 0.1704 ± 0.0572
   Rango BLEU:   [0.0046, 0.0210]
   Rango METEOR: [0.0912, 0.2787]
   Muestras: 15

ADVANCED
   BLEU:   0.0091 ± 0.0086
   METEOR: 0.1457 ± 0.0449
   Rango BLEU:   [0.0011, 0.0304]
   Rango METEOR: [0.0803, 0.2246]
   Muestras: 15

PROMEDIO GENERAL (TODOS LOS NIVELES)
BLEU:   0.0284 ± 0.0595
METEOR: 0.2312 ± 0.1624
Total: 45 muestras



In [11]:
print("="*70)
print("EXTRAYENDO MÉTRICAS")
print("="*70 + "\n")

detailed_metrics = []
metrics_by_level = {
    "basic": {"score_a": [], "score_b": []},
    "intermediate": {"score_a": [], "score_b": []},
    "advanced": {"score_a": [], "score_b": []}
}

for image_data in results:
    image_name = image_data["image_path"]
    annotations = image_data["annotations"]
    prompt_outputs = image_data["prompt_outputs"]

    print(f"{image_name}")

    image_metrics = {
        "image": image_name,
        "levels": {}
    }

    for i, output_data in enumerate(prompt_outputs):
        prompt = output_data["prompt"]
        generated = output_data["output"]
        level = prompt_to_level[prompt]
        reference = annotations[i]

        # Extraer métricas ya calculadas
        score_a = output_data["scores"]["score_a"]
        score_b = output_data["scores"]["score_b"]
        execution_time = output_data.get("execution_time", 0)

        image_metrics["levels"][level] = {
            "reference": reference,
            "generated": generated,
            "score_a": score_a,
            "score_b": score_b,
            "execution_time": execution_time
        }

        # Agregar a estadísticas
        metrics_by_level[level]["score_a"].append(score_a)
        metrics_by_level[level]["score_b"].append(score_b)

        print(f"  {level:12} | Score A: {score_a:.4f} | Score B: {score_b:.4f}")

    detailed_metrics.append(image_metrics)
    print()

EXTRAYENDO MÉTRICAS

01.jpg
  basic        | Score A: 0.6667 | Score B: 0.9212
  intermediate | Score A: 0.3000 | Score B: 0.8715
  advanced     | Score A: 0.4068 | Score B: 0.9070

02.jpg
  basic        | Score A: 0.2857 | Score B: 0.9107
  intermediate | Score A: 0.1509 | Score B: 0.8737
  advanced     | Score A: 0.1724 | Score B: 0.8875

03.jpg
  basic        | Score A: 0.2286 | Score B: 0.9054
  intermediate | Score A: 0.2174 | Score B: 0.8787
  advanced     | Score A: 0.2000 | Score B: 0.8776

04.jpg
  basic        | Score A: 0.4800 | Score B: 0.9159
  intermediate | Score A: 0.2258 | Score B: 0.8813
  advanced     | Score A: 0.2353 | Score B: 0.8973

05.jpg
  basic        | Score A: 0.7778 | Score B: 0.9689
  intermediate | Score A: 0.1702 | Score B: 0.8822
  advanced     | Score A: 0.2623 | Score B: 0.8837

06.jpg
  basic        | Score A: 0.5263 | Score B: 0.9456
  intermediate | Score A: 0.2500 | Score B: 0.8868
  advanced     | Score A: 0.4074 | Score B: 0.9177

07.jpg
  basi